# Homework Assignment 3

## Data Processing in R & Python
## Emilia Wróblewska

## Loading packages and data
Firstly, import all required modules and open the database connection.
(Note: The .ipynb file must be placed in the same directory as packed .csv files with data and requires `pandas` and `numpy` packages to be already installed.)

In [143]:
import numpy as np
import pandas as pd
import tempfile
import sqlite3
import os

Votes = pd.read_csv("Votes.csv.gz", compression='gzip')
Posts = pd.read_csv("Posts.csv.gz", compression='gzip')
Users = pd.read_csv("Users.csv.gz", compression='gzip')

# path to database file
database = os.path.join(tempfile.mkdtemp(), 'example.db')
if os.path.isfile(database): # remove if this file already exists
    os.remove(database)

connection = sqlite3.connect(database) # create the connection
Posts.to_sql("Posts", connection)
Users.to_sql("Users", connection)
Votes.to_sql("Votes", connection)

Through the entire report I will be using the comparison function from `pandas` package `.equals()` to compare the results of my solutions with the reference results to ensure the correctness of obtained answers.

## SQL Query 2
The first query I decided to implement is query number 2, which returns 10 most frequently occurring locations of users who shared any Post, along with their number of occurences.

### Reference solution

In [144]:
ref_solution_2 = pd.read_sql_query(
    """
    SELECT Location, COUNT(*) AS Count
    FROM (
        SELECT Posts.OwnerUserId, Users.Id, Users.Location
        FROM Users
        JOIN Posts ON Users.Id = Posts.OwnerUserId
        )
    WHERE Location NOT IN ('')
    GROUP BY Location
    ORDER BY Count DESC
    LIMIT 10
    """, 
    connection)
ref_solution_2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


### Pandas solution

In [145]:
# Firstly, join Users and Posts data frames and extract only needed columns
joined = pd.merge(Users[["Id","Location"]], Posts["OwnerUserId"], left_on = 'Id', right_on = 'OwnerUserId')

# Then, get rows with non-empty Location, group by Location and count 
grouped = joined.dropna(subset = ["Location"]).groupby(["Location"]).size().reset_index(name = 'Count')

# Sort by Count column in descending order and select top 10 rows to get final answer
pd_solution_2 = grouped.sort_values(by = ['Count'], ascending = False).reset_index(drop = True).head(10)
pd_solution_2

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


### Accuracy test

In [146]:
pd_solution_2.equals(ref_solution_2)

True

### Comments
This query was quite easy to implement as it required only one join of tables and one grouping. The functions from `pandas` are very intuitive to use, especially after solving the first project in R. I applied a very similar logic as in the `data.table` solution - firstly, I joined the tables and limited the number of columns, then filtered, grouped and counted the records to get the final answer. 

## SQL Query 3
The second query I decided to implement is query number 3, which displays the information (`AccoutnId`, `DisplayName` and `Location`) about first 10 users with the highest average number of answers to their posts.

### Reference solution

In [147]:
ref_solution_3 = pd.read_sql_query(
    """
    SELECT
        Users.AccountId,
        Users.DisplayName,
        Users.Location,
        AVG(PostAuth.AnswersCount) as AverageAnswersCount
    FROM
        (
        SELECT
            AnsCount.AnswersCount,
            Posts.Id,
            Posts.OwnerUserId
        FROM 
            (
                SELECT Posts.ParentId, COUNT(*) AS AnswersCount
                FROM Posts
                WHERE Posts.PostTypeId = 2
                GROUP BY Posts.ParentId
            ) AS AnsCount
            JOIN Posts ON Posts.Id = AnsCount.ParentId
        ) AS PostAuth
    JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
    GROUP BY OwnerUserId
    ORDER BY AverageAnswersCount DESC
    LIMIT 10
    """, 
    connection)
ref_solution_3

,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,44093.0,Emma Arbogast,"Salem, OR",10.0
3,204.0,Josh,Australia,10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,79346.0,Thomas Matthews,California,8.0
6,54571.0,Christian,None,8.0
7,42364.0,Petrogad,None,8.0
8,20473.0,Jeremy Boyd,"Houston, TX",8.0
9,19588.0,JD Isaacks,"Atlanta, GA",8.0


### Pandas solution

In [148]:
# Get the first table - AnsCount - by counting answers under each post in Posts table
AnsCount = Posts[Posts["PostTypeId"] == 2].groupby(["ParentId"]).size().reset_index(name = 'AnswersCount')

# Merge AnsCount and Posts, select appropriate columns and merge the result with Users table
PostAuth = pd.merge(Posts, AnsCount, left_on = 'Id', right_on = 'ParentId')[["AnswersCount","Id","OwnerUserId"]]
joined = pd.merge(Users, PostAuth, left_on  = 'AccountId', right_on = 'OwnerUserId')

# Calculate the mean of AnswersCount per post owner
avgAnswers = joined.groupby(["OwnerUserId"]).mean().reset_index()

# Since I lost some columns while applying mean(), I join the appropriate columns from previous data frame back
tmp = pd.merge(avgAnswers[["OwnerUserId","AnswersCount"]], joined[["DisplayName","Location","AccountId"]], 
                  left_on = 'OwnerUserId', right_on = 'AccountId')

# Clean the final result - sort values, rename and select needed columns 
pd_solution_3 = tmp.sort_values(by = ['AnswersCount','AccountId'], ascending = [False, False]).reset_index(drop = True).head(10)
pd_solution_3 = pd_solution_3.rename(columns = {'AnswersCount': 'AverageAnswersCount'})
pd_solution_3 = pd_solution_3[["AccountId","DisplayName","Location","AverageAnswersCount"]]
pd_solution_3

,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,44093.0,Emma Arbogast,"Salem, OR",10.0
3,204.0,Josh,Australia,10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,79346.0,Thomas Matthews,California,8.0
6,54571.0,Christian,NaN,8.0
7,42364.0,Petrogad,NaN,8.0
8,20473.0,Jeremy Boyd,"Houston, TX",8.0
9,19588.0,JD Isaacks,"Atlanta, GA",8.0


### Accuracy test

In [149]:
pd_solution_3.equals(ref_solution_3)

True

### Comments
This query was a bit harder than the first one, since the aggregation and joining in SQL solution is done at the same time, which is not so easily done in `pandas`. However, I solved this problem by firstly calculating the mean value of `AnswersCount` in each group and then joining the resulting table with the desired columns from the previous filtered table. The drawback of this solution is the need to do some cleaning in the end, i.e. selecting relevant columns and renaming some of them.

## SQL Query 4
The third query I decided to implement is query number 4, which shows the title of the post which has obtained the most up votes during each recorded year along with the calculated number of up votes.

### Reference solution

In [150]:
ref_solution_4 = pd.read_sql_query(
    """
    SELECT
        Posts.Title,
        UpVotesPerYear.Year,
        MAX(UpVotesPerYear.Count) AS Count
    FROM 
        (
        SELECT
            PostId,
            COUNT(*) AS Count,
            STRFTIME('%Y', Votes.CreationDate) AS Year
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, Year
        ) AS UpVotesPerYear
    JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
    WHERE Posts.PostTypeId=1
    GROUP BY Year
    ORDER BY Year ASC
    """, 
    connection)
ref_solution_4

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,179
4,Immigration officer that stopped me at the air...,2015,117
5,I don't know my nationality. How can I visit D...,2016,134
6,Why prohibit engine braking?,2017,177
7,How can I find restaurants in the USA where ti...,2018,119
8,My name causes an issue with any booking! (nam...,2019,263
9,What's the longest distance that can be travel...,2020,110


### Pandas solution

In [151]:
# Extract year from CreationDate column and count PostIds per Year
Votes['Year'] = Votes['CreationDate'].str.slice(start = 0, stop = 4)
UpVotesPerYear = Votes[Votes['VoteTypeId'] == 2].groupby(['PostId','Year']).size().reset_index(name = 'Count')

# Join UpVotesPerYear with Posts table and appropriately filter the result
joined = pd.merge(Posts, UpVotesPerYear, left_on  = 'Id', right_on = 'PostId')
titles = joined[joined['PostTypeId'] == 1][['Title','Year','Count']]

# Get the Title with max Count value in each Year by sorting the titles table and extracting first record for each group
pd_solution_4 = titles.sort_values(['Year', 'Count'], ascending=[True, False]).groupby('Year').first().reset_index()
pd_solution_4 = pd_solution_4.reindex(columns = ['Title','Year','Count'])
pd_solution_4

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,179
4,Immigration officer that stopped me at the air...,2015,117
5,I don't know my nationality. How can I visit D...,2016,134
6,Why prohibit engine braking?,2017,177
7,How can I find restaurants in the USA where ti...,2018,119
8,My name causes an issue with any booking! (nam...,2019,263
9,What's the longest distance that can be travel...,2020,110


### Accuracy test

In [152]:
pd_solution_4.equals(ref_solution_4)

True

In [153]:
# Close the database connection
connection.close()

### Comments
This query was much easier for me to solve using `pandas` package than the R approach. Firstly, similarly to as I solved this query in R, I decided to extract the `Year` from `CreationDate` column by taking advantage of the operations on strings and use `str.slice()`. This way, the solution is much faster and I avoid the double conversion from string to date and back to string again. And secondly, opposite to the implemetation of the R solution of this query, finding maximum value for each group turned out to be very easy with `pandas`. After filtering out relevant data, I simply sorted the values in descending order by `Year` and `Count` columns and extracted the first value for each group, which would obviously be the maximum.

## Summary
Although I didn't have any previous experience with `pandas`, I already had some in Python and `numpy` package and I think it made writing the solution much easier. The queries were the same as in the first project, so I only had to recall the logic I used there and rewrite it using `pandas` functions, that, as it turned out, are very similar to their R counterparts.

The only differences I observed were the indexes, which needed to be reset every time I sorted or aggregated data frames, and the Series data type. I also noticed that the merging and aggregating functions were much faster in `pandas` than they were in R, which was very nice. Overall, I think I enjoyed implementing the solutions in `pandas` more, because it was way faster and thanks to the *Jupyter Notebook* being a very convenient tool, much easier and clear to use than the *Knitr* reports.